In [1]:
import os
import logging, timeit
from btEngine2.DataLoader import DataLoader
from btEngine2.MarketData import MarketData
from btEngine2.TradingRule import TradingRule



# Define paths and configurations for DataLoader
ticker_csv_path = r'G:\Projects\BackTesting1.0\Data\Inputs\TickerList-Futs.csv'
save_directory = r"G:\Projects\BackTesting1.0\Data\Bloomberg\Futures"
helper_directory = r'G:\Projects\BackTesting1.0\Data\Bloomberg\HelperFiles'

bt_folder = r'BackTests'

# Define paths to auxiliary data for MarketData
tick_values_path = os.path.join(helper_directory, 'fut_val_pt.parquet')
fx_rates_path = os.path.join(helper_directory, 'fxHist.parquet')

# Initialize the MarketData
market_data = MarketData(
    base_directory=save_directory,
    tick_values_path=tick_values_path,
    fx_rates_path=fx_rates_path,
    instrument_type="Futures",
    n_threads=8,  # Number of threads for parallel data loading
    log_level=logging.ERROR  # Set to DEBUG for more detailed logs
)


In [ ]:
tick = 'ES1 Index'
# Access data for a specific ticker
try:
    test_df = market_data.get_ticker_data(tick)
    print(test_df)
except ValueError as e:
    print(e)

# Access all preprocessed data
all_data = market_data.get_data()
print(f"Total tickers loaded: {len(all_data)}")

# Access FX rates
fx_rates = market_data.get_fx_rates()
# Access tick values
tick_values = market_data.get_tick_values()
# Access asset classes
asset_classes = market_data.get_asset_classes()

#market_data = market_data.date_filter(start_date='01012010')

In [3]:

pSizeParams = {
    'AssetVol': r'G:\Projects\BackTesting1.0\Data\Inputs\AssetSizing-Futs.csv',  # Target asset volatility in USD
    'VolLookBack': 21  # Lookback period for volatility calculation
}


In [4]:
from btEngine2.Rules.MeanReversion.rsi_mr import *


strategy_params_l = {
    'N': 2,
    'lmt_atr_ratio': 0.7,
    'lmt_atr_period': 3,
    'lmt_epsilon':0.1,
    'rsi_period': 3,
    'consecutive_days': 3,
    'rsi_threshold': 5.0,  # Typically low RSI for long entries
    'optional_rsi': (3, 2, 25),    # (rsi_period, cumulative_days, threshold)
    'optional_ma': None,     # (ma_period, ma_type)
    'atr_type':'atr'
}

strategy_params_s = {
    'N': 2,
    'lmt_atr_ratio': 0.7,
    'lmt_atr_period': 3,
    'lmt_epsilon':0.1,
    'rsi_period': 3,
    'consecutive_days': 3,
    'rsi_threshold': 5.0,  # Typically low RSI for long entries
    'optional_rsi': (3, 2, 75),    # (rsi_period, cumulative_days, threshold)
    'optional_ma': None,     # (ma_period, ma_type)
    'atr_type':'atr'
}

#strategy_params_l = {}
#strateg_params_s = {}

In [5]:
### Where the magic happens
mr_long = TradingRule(
    market_data=market_data,
    trading_rule_function=rsi_mrb_long,
    trading_params=strategy_params_l,
    position_sizing_params=pSizeParams,  # Define as needed
    name_label='ts_rsimr_long',
    log_level=logging.ERROR,
    bt_folder = bt_folder
)

mr_short = TradingRule(
    market_data=market_data,
    trading_rule_function=rsi_mrb_short,
    trading_params=strategy_params_s,
    position_sizing_params=pSizeParams,  # Define as needed
    name_label='ts_rsimr_short',
    log_level=logging.ERROR,
    bt_folder = bt_folder
)

In [39]:


res = mr_long.backtest_all_assets(save=True)
ress = mr_short.backtest_all_assets(save=True)

In [ ]:
filtr = asset_classes['fi-us'] + asset_classes['fi-eu'] + asset_classes['stir'] + asset_classes['fi-as']
resdb = mr_long.plot_equity(byac=True,totalsys=True, start_date = '01012000', save_fig=False, filter_assets=filtr)


In [ ]:
resdb = mr_short.plot_equity(byac=True,totalsys=True, start_date = '01012000', save_fig=False, filter_assets=filtr)


In [ ]:
test_df['Close'].diff()

In [ ]:
compute_effective_atr(test_df, 14)

In [12]:
testing = rsi_mrb_long(test_df, **strategy_params_l)
testing.to_pandas().to_clipboard()